In [ ]:
from pyArango.connection import *
from IPython.display import clear_output
import getpass
import pandas as pd

In [ ]:
username = input('Please enter your username to connect DB: ')
password = getpass.getpass('Please enter your password to connect DB: ')
try:
    conn = Connection(username=username, password=password)
    clear_output(wait=True)
    print('Successfully connected to the ArangoDB')
    del password
except ConnectionError:
    clear_output(wait=True)
    print('Warning: Please check your credentials and try to connect again!')
    del password

In [57]:
db = conn["Flemish_Business_Websites"]
#BusinessCollection = db.createCollection(name="Businesses")
BusinessCollection = db["Businesses"]

In [8]:
FILE = 'C:\\Users\\nusret\\Desktop\\Thesis\\data\\inno5_address_toStatVla_cis19.xlsx'
CIS_Survey_Dataframe = pd.read_excel(FILE, sheet_name='inno5_address_toStatVla_cis19')

In [ ]:
CIS_Survey_Dataframe

In [66]:
for index, row in CIS_Survey_Dataframe.iterrows():
    doc = BusinessCollection.createDocument()
    doc['Name'] = row['cur_firm']
    if not (pd.isnull(row["gpnameStata19"])):    
        doc['Group'] = row["gpnameStata19"]
    if not (pd.isnull(row["inno5"])):    
        doc['Innovation'] = int(row["inno5"])
    if not (pd.isnull(row["cur_street1"])):    
        doc['Street_1'] = row["cur_street1"]        
    if not (pd.isnull(row["cur_street2"])):    
        doc['Street_2'] = row["cur_street2"]
    if not (pd.isnull(row["finalzip"])):    
        doc['Main_Zip'] = row["finalzip"]  
    if not (pd.isnull(row["zipActivPost19"])):    
        doc['Activity_Zip'] = row["zipActivPost19"]   
    if not (pd.isnull(row["cur_email"])):    
        doc['Registry_Email'] = row["cur_email"]      
    if not (pd.isnull(row["cur_web"])):    
        doc['Registry_Website'] = row["cur_web"]          
    if not (pd.isnull(row["email"])):    
        doc['User_Provided_Email'] = row["email"]      
    if not (pd.isnull(row["web"])):    
        doc['User_Provided_Website'] = row["web"]        
    if not (pd.isnull(row["street1"])):    
        doc['User_Provided_Street_1'] = row["street1"]        
    if not (pd.isnull(row["street2"])):    
        doc['User_Provided_Street_2'] = row["street2"]            
    doc._key = str(row['btw'])
    doc.save()

In [74]:
for b in BusinessCollection.fetchAll():
    if b['Group'] is not None:
        print(b['Group'].ljust(30), b['Innovation'])

vpk                            None
tupperware                     1
deme                           1
dossche                        1
lotus                          1
elep                           1
plakabeton                     1
xeikon                         1
aveve                          1
ico                            1
fagron                         1
creatuft                       1
Niko                           1
willemen                       None
milcobel                       1
depuydt                        1
bekaertdeslee                  1
vpk                            None
cras                           1
vpk                            None
unilin                         1
cgc                            1
bekaertdeslee                  1
remic                          1
orac                           1
vincotte                       1
vandezande                     1
transa                         0
agidens                        1
daikin                         

In [75]:
db.AQLQuery("RETURN COUNT(FOR b IN Businesses RETURN 1)", rawResults=True)[0]

3179

In [ ]:
FILE = 'Final_URLs.csv'
Found_URLS = pd.read_csv(FILE, sep=';')
Found_URLS.head()

In [82]:
for index, row in Found_URLS.iterrows():
    doc = BusinessCollection[str(row["btw"])]
    doc["Final_Website"] = row["New Best Guess"]
    doc.save()

In [ ]:
BusinessCollection['200068636']

In [99]:
print("Total Websites Found:".ljust(25), db.AQLQuery("RETURN COUNT(FOR b IN Businesses FILTER b.Final_Website != NULL RETURN 1)", rawResults=True)[0])
print("Total businesses in CIS:".ljust(25), db.AQLQuery("RETURN COUNT(FOR b IN Businesses RETURN 1)", rawResults=True)[0])
print("Website Coverage %:".ljust(25), round(((db.AQLQuery("RETURN COUNT(FOR b IN Businesses FILTER b.Final_Website != NULL RETURN 1)", rawResults=True)[0])/(db.AQLQuery("RETURN COUNT(FOR b IN Businesses RETURN 1)", rawResults=True)[0]))*100, 2), "%")

Total Websites Found:     3005
Total businesses in CIS:  3179
Website Coverage %:       94.53 %
